# Comparative Analysis of REGEX v. Pattern Matching Algorithms

___

## Approaches
#### Regex
Regex engines internally use optimized algorithms like Finite State Automata (FSA) or Thompson's NFA to match patterns of varying complexity, from fixed strings to complex rules (e.g., optionality, repetition). 
Regex processes the entire string using internal algorithms without explicitly managing a "window."  Consequently, time complexity depends on the regex pattern and engine design:
For simple patterns: O(n)) with linear scans.
For complex patterns: With excessive backtracking, worst-case performance can degenerate to O(2n).

#### Algorithms
We will review the following algorithms:
*  Knuth Morris Pratt
*  Rabin Karp
*  Boyer Moore
*  Wu Manber

Sliding Window Optimization and Heuristics
The sliding window optimization involves preprocessing the pattern or string and using a ‘window’ of elements to incrementally move through the string and reusing information to reduce unnecessary comparisons.  
The sliding window algorithms achieve efficiency by with Optimization, where hash values are updated as the window slides (e.g., Rabin-Karp), and Heuristics, using precomputed bad character or good suffix tables (e.g., Boyer-Moore).
Trie Algorithms
A trie approach builds a tree–like data structure to store a set of strings where nodes represent characters and paths represent sets of strings. 
Tire algorithms are efficient for multi-pattern searching and prefix-based search of large datasets


## Other String Search & Pattern Matching Algorithms
There are other important algorithms which are not dealth with in this workbook
#### Aho–Corasick Algorithm
Introduced in 1975, the Aho–Corasick algorithm is a multi-pattern string matching algorithm designed to efficiently find instances of multiple patterns in a text simultaneously. It constructs a trie structure of the patterns, augmented with failure links to handle mismatches and allows for seamless transitions between patterns during the search phase. This preprocessing results in a time complexity of O(n+m+k), where n is the text length, m is the total length of all patterns, and k is the number of matches. This algorithm is widely used in applications like text searching, intrusion detection, and bioinformatics.
#### Commentz-Walter Algorithm
Developed in 1979, the Commentz-Walter algorithm extends the Boyer-Moore algorithm to handle multiple pattern searches efficiently. It combines the concepts of the Boyer-Moore algorithm for individual patterns and the Aho–Corasick algorithm for managing multiple patterns, building a trie structure with precomputed shift tables. This hybrid approach allows for rapid skipping of text sections during mismatches while accommodating multiple patterns. Its time complexity is O(n+m+k), where n is the text length, m is the total length of all patterns, and k is the number of matches, making it suitable for high-performance multi-pattern searching tasks.

___

# String Search with REGEX

In [1]:
import re
import time  # Import the time library
from collections import Counter

Regexes are widely supported in programming languages (Java, Python, Perl) and text processing tools.  Developed in the 1950s by Stephen Cole Kleene for describing regular languages in theoretical computer science, during the 1960's-70's, REGEX was implemented in Unix tools like QED and grep and became integral to text editors and compilers. Regex engines wuse Finite State Automatons (FSA) or Thompson's NFA for matching patterns with performance of O(n) to O(2n).  In contrast to window sliding algorithms, REGEX processes entire the entire string and Features like lookahead/lookbehind assertions, backreferences, and lazy quantifiers allow REGEX to match strings of various complex text processing tasks from exact string search to multi-pattern matching with wildcards and optional characters.  

In [2]:
# Regex pattern matching solution 

def file_opener(file_path: str) -> str:

    try:
        start_time = time.time()
        word_count = 0
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
            match = r"\w+"
            for match in re.finditer(match, content):
                word_count += 1
            print(f"There are {word_count} words in the text '{file_path}'")
        elapsed_time = time.time() - start_time  # Calculate elapsed time
        print(f"File opened and read in {elapsed_time:.4f} seconds.")
        return content

    except Exception as e:
        print(f"A file error occurred: {e}")


def word_count_summary(file_path: str, search_terms: list[str]) -> None:

    start_time = time.time()  
    content = file_opener(file_path)

    try:
        if type(search_terms) == str:  # Input is string, not list
            pattern = r'\b' + search_terms + r'\b'
            frequency = len(re.findall(pattern, content, re.IGNORECASE))
            print(f"The word '{search_terms}' appears {frequency} times.\n")
        else:
            print(f"The {len(search_terms)} search terms appear in this text as follows: \n")  # Input is a list, not string

            # Dynamic formatting for table
            longest_term = max((term.strip() for term in search_terms), key=len)
            column_width = len(longest_term) if len(longest_term) > 8 else 8
            line_break = f"| {'-' * column_width} | {'-' * column_width} |"
            header = "| {0:^{1}} | {2:^{1}} |".format('Word', column_width, 'Count')
            print(f"{line_break}\n{header}\n{line_break}")

            # Strip terms for multi-term search
            word_counter = 0
            for word in search_terms:
                word = word.strip()
                pattern = r'\b' + word + r'\b'
                frequency = len(re.findall(pattern, content, re.IGNORECASE))
                word_counter += frequency
                print(f"| {word:<{column_width}} | {frequency:>{column_width}} |")  # Output aligned left/right by column

            sum_total = f"| {'Total':<{column_width}} | {word_counter:>{column_width}} |"
            print(f"{line_break}\n{sum_total}\n{line_break}\n")
    except Exception as e:
        print(f"A counter error occurred: {e}")
    
    elapsed_time = time.time() - start_time  # Calculate elapsed time for word count summary
    print(f"Word count summary completed in {elapsed_time:.4f} seconds.")

def main():
    start_time = time.time()  # Start the overall timer

    file_path = 'a-tale-of-two-cities.txt'
    search_terms = ['birds', 'women', 'horses']
    word_count_summary(file_path, search_terms)

    elapsed_time = time.time() - start_time  # Overall execution time
    print(f"\nTotal execution time: {elapsed_time:.4f} seconds")

if __name__ == '__main__':
    main()


There are 138206 words in the text 'a-tale-of-two-cities.txt'
File opened and read in 0.0431 seconds.
The 3 search terms appear in this text as follows: 

| -------- | -------- |
|   Word   |  Count   |
| -------- | -------- |
| birds    |       10 |
| women    |       61 |
| horses   |       40 |
| -------- | -------- |
| Total    |      111 |
| -------- | -------- |

Word count summary completed in 0.0902 seconds.

Total execution time: 0.0904 seconds


___

# Knuth-Morris-Pratt (KMP) Algorithm
## Sliding Window String Search with Optimization  

Introduced in 1977, the Knuth-Morris-Pratt (KMP) algorithm is a linear-time string matching algorithm designed to efficiently locate substrings within a text. It avoids redundant comparisons by preprocessing the pattern to create a Longest Proper Prefix which is also a Suffix (LPS) table. This table allows the algorithm to skip unnecessary re-evaluations of already matched characters when a mismatch occurs. During the search phase, the algorithm leverages the LPS table to align the pattern dynamically, reducing the overall number of comparisons. With a time complexity of O(n + m), where n is the length of the text and m is the length of the pattern, KMP is particularly effective for scenarios with repetitive patterns or substrings, though it is less suited for handling complex pattern matching tasks.

In [3]:
# Acknowledgement: Code adapted from  
# https://gist.github.com/m00nlight/daa6786cc503fde12a77#gistcomment-2750908

def compute_lps_array(pattern: str) -> list[int]:
    """
    Computes the Longest Prefix Suffix (LPS) array for the KMP algorithm.
    """
    lps = [0] * len(pattern)
    length = 0  # Length of the previous longest prefix suffix
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps


def kmp_search(content: str, pattern: str) -> int:
    """
    Searches for a pattern in the given content using the KMP algorithm.
    Returns the frequency of the pattern in the content.
    """
    n = len(content)
    m = len(pattern)
    lps = compute_lps_array(pattern)
    
    i = 0  # Index for content
    j = 0  # Index for pattern
    match_count = 0  # Count of matches
    word_comparisons = 0  # Counter for comparisons

    while i < n:
        word_comparisons += 1  # Increment comparisons counter
        if content[i] == pattern[j]:
            i += 1
            j += 1

        if j == m:
            match_count += 1
            j = lps[j - 1]  # Reset j using the LPS array
        elif i < n and content[i] != pattern[j]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    return match_count, word_comparisons


def word_count_summary_kmp(file_path: str, search_terms: list[str]) -> None:
    """
    Word frequency analysis with KMP algorithm.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
        
        print(f"Searching in file: {file_path}")
        start_time = time.time()  # Start timing
        total_comparisons = 0  # Counter for total comparisons across all search terms

        print("\nResults:")
        for term in search_terms:
            frequency, comparisons = kmp_search(content, term)
            total_comparisons += comparisons
            print(f"'{term}' appears {frequency} times. Comparisons: {comparisons}")

        elapsed_time = time.time() - start_time
        print(f"\nTotal comparisons: {total_comparisons}")
        print(f"Search completed in {elapsed_time:.4f} seconds.")

    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except Exception as e:
        print(f"An error occurred: {e}")


if __name__ == "__main__":
    file_path = "a-tale-of-two-cities.txt"
    search_terms = ["birds", "women", "horses"]
    word_count_summary_kmp(file_path, search_terms)


Searching in file: a-tale-of-two-cities.txt

Results:
'birds' appears 10 times. Comparisons: 755864
'women' appears 61 times. Comparisons: 755864
'horses' appears 40 times. Comparisons: 755864

Total comparisons: 2267592
Search completed in 0.3477 seconds.


___

# Boyer-Moore Algorithm  
## Sliding Windome String Search with Heuristics

Proposed in 1977, the Boyer-Moore algorithm is a string matching algorithm that achieves high efficiency by skipping sections of the text during the search phase. It leverages two heuristics: the bad character rule and the good suffix rule, which determine how far the pattern can be shifted when a mismatch occurs. These rules significantly reduce the number of comparisons, especially in practical scenarios where the pattern is long or the alphabet is large. The algorithm’s worst-case time complexity is O(n ⋅ m) but it performs close to O(n / m) on average for random text and patterns.

In [4]:
# Acknowledgement: Code is adapted from
# https://github.com/je-suis-tm/search-and-sort/blob/master/boyer%20moore%20search.py

def boyer_moore_search(content, pattern):
  """
  Boyer-Moore algorithm for single pattern matching.
  """
  n = len(content)
  m = len(pattern)
  if m == 0 or m > n:
      return []

  # Preprocess the pattern to create the bad character heuristic table
  bad_char = dict()  # Create a dictionary for bad character heuristic
  for i in range(m):
      bad_char[ord(pattern[i])] = i

  matches = []
  shift = 0

  # Slide the pattern over the content
  while shift <= n - m:
      j = m - 1

      # Match pattern from the end
      while j >= 0 and content[shift + j] == pattern[j]:
          j -= 1

      if j < 0:
          # Pattern found at this shift
          matches.append(shift)
          # Move the pattern to the next possible position
          shift += (m if (shift + m) < n else 1)  # Move by the full length or 1 if we're at the end of the text
      else:
          # Use the bad character heuristic to shift
          bad_char_shift = bad_char.get(ord(content[shift + j]), -1)  # Handle potential out-of-bounds access using get method of dictionary
          shift += max(1, j - bad_char_shift)

  return matches

def count_words(text):
    return len(text.split())

if __name__ == "__main__":
    # Read the text from a file
    file_name = "a-tale-of-two-cities.txt"
    with open(file_name, "r", encoding="utf-8") as file:
        text = file.read()

    patterns = ["birds", "women", "horses"]

    total_words = count_words(text)
    print(f"Total words in the text: {total_words}")

    # Run Boyer-Moore search for each pattern
    print("Running Boyer-Moore Search...")
    total_matches = Counter()
    total_time = 0

    for pattern in patterns:
        start_time = time.time()
        matches = boyer_moore_search(text, pattern)
        end_time = time.time()

        total_time += (end_time - start_time)
        total_matches[pattern] += len(matches)

        print(f"Pattern '{pattern}' found {len(matches)} time(s).")

    print(f"\nTotal time taken for Boyer-Moore search: {total_time:.6f} seconds")
    print("Pattern counts:")
    for pattern, count in total_matches.items():
        print(f"  '{pattern}': {count} occurrence(s)")

Total words in the text: 135660
Running Boyer-Moore Search...
Pattern 'birds' found 10 time(s).
Pattern 'women' found 61 time(s).
Pattern 'horses' found 40 time(s).

Total time taken for Boyer-Moore search: 0.120899 seconds
Pattern counts:
  'birds': 10 occurrence(s)
  'women': 61 occurrence(s)
  'horses': 40 occurrence(s)


___

# Rabin-Karp Algorithm

Developed in 1987, the Rabin-Karp algorithm is a string matching algorithm that utilizes hashing to efficiently search for patterns within a text. It works by calculating a hash value for the pattern and each substring of the text of the same length, allowing for quick identification of potential matches. In case of a hash match, the algorithm verifies the actual characters to confirm the match, ensuring correctness. While its average-case time complexity is O(n+m), where n is the text length and m is the pattern length, its worst-case complexity is O(n⋅m), particularly if hash collisions occur frequently.

In [5]:
def rabin_karp_search(text, pattern, prime=101):
    """
    Rabin-Karp algorithm for pattern matching.
    """
    n = len(text)
    m = len(pattern)
    d = 256  # Number of characters in the input alphabet
    h = 1
    p = 0  # Hash value for the pattern
    t = 0  # Hash value for the current substring in the text
    result = []

    # Compute h = d^(m-1) % prime
    for _ in range(m - 1):
        h = (h * d) % prime

    # Compute initial hash values for pattern and first window of text
    for i in range(m):
        p = (d * p + ord(pattern[i])) % prime
        t = (d * t + ord(text[i])) % prime

    # Slide the pattern over the text one character at a time
    for i in range(n - m + 1):
        # Check if hash values match
        if p == t:
            # Verify characters one by one
            if text[i:i + m] == pattern:
                result.append(i)

        # Compute hash for the next substring
        if i < n - m:
            t = (d * (t - ord(text[i]) * h) + ord(text[i + m])) % prime

            # Handle negative hash value
            if t < 0:
                t += prime

    return result

def count_words(text):
    return len(text.split())

if __name__ == "__main__":
    file_name = "a-tale-of-two-cities.txt"
    with open(file_name, "r", encoding="utf-8") as f:
        text = f.read()

    patterns = ['birds', 'women', 'horses']

    word_count = count_words(text)
    print(f"Total words in the text: {word_count}")

    for pattern in patterns:
        print(f"Searching for pattern: '{pattern}'")
        start_time = time.time()
        matches = rabin_karp_search(text, pattern)
        end_time = time.time()

        print(f"Occurrences of '{pattern}': {len(matches)}")
        elapsed_time = end_time - start_time
        print(f"Time taken for Rabin-Karp search: {elapsed_time:.6f} seconds\n")
        


Total words in the text: 135660
Searching for pattern: 'birds'
Occurrences of 'birds': 10
Time taken for Rabin-Karp search: 0.174882 seconds

Searching for pattern: 'women'
Occurrences of 'women': 61
Time taken for Rabin-Karp search: 0.172136 seconds

Searching for pattern: 'horses'
Occurrences of 'horses': 40
Time taken for Rabin-Karp search: 0.173486 seconds



___

# Wu-Manber Algorithm

Proposted in 1994, Wu-Manber algorithm is highly useful for finding words or patterns in texts, especially when searching for multiple patterns simultaneously. It is a multi-pattern string matching algorithm designed for high-speed searching in large-scale texts where efficiency is critical. it is optimized for exact string matching but can be adapted for approximate matching with small modifications. Similar to the Boyer-Moore algorithm, it leverages hash tables and shift tables to handle multiple patterns simultaneously. The algorithm preprocesses the patterns to compute a shift table and uses a hashing mechanism to identify candidate matches quickly. Importantly, it minimizes the amount of text scanned during mismatches, making it particularly well-suited for applications like text indexing, network intrusion detection, and digital forensics. The Wu-Manber algorithm achieves a time complexity of 𝑂(𝑛) on average, where 𝑛 is the text length.

In [6]:
import time
from collections import Counter

def wu_manber_search(content, patterns):
    """
    Wu-Manber search algorithm for multiple pattern matching
    """
    # Step 1: Preprocessing - Build the hash table
    shift_table = {}
    min_pattern_length = min(len(pattern) for pattern in patterns)

    # Populate shift table for all patterns
    for pattern in patterns:
        for i in range(len(pattern) - min_pattern_length + 1):
            substring = pattern[i:i + min_pattern_length]
            if substring not in shift_table:
                shift_table[substring] = []
            shift_table[substring].append(pattern)

    # Step 2: Search in text
    matches = []
    i = 0
    while i <= len(content) - min_pattern_length:
        substring = content[i:i + min_pattern_length]
        if substring in shift_table:
            # Verify each candidate pattern
            for candidate in shift_table[substring]:
                if content[i:i + len(candidate)] == candidate:
                    matches.append((i, candidate))
        i += 1

    return matches

def count_words(text):
    return len(text.split())

# Example Usage with Timer and Pattern Count
if __name__ == "__main__":
    file_name = "a-tale-of-two-cities.txt"
    with open(file_name, "r", encoding="utf-8") as file:
        text = file.read()

    patterns = ["birds", "women", "horses"]

    total_words = count_words(text)
    print(f"Total words in the text: {total_words}")

    # Run Wu-Manber search
    start_time = time.time()  # Start timer
    results = wu_manber_search(text, patterns)
    end_time = time.time()  # End timer

    # Count occurrences of each pattern
    pattern_counts = Counter([match[1] for match in results])

    # Print results
    if results:
        for pattern in patterns:
            count = pattern_counts.get(pattern, 0)
            print(f"Pattern '{pattern}' found {count} time(s).")
    else:
        print("No patterns found.")

    # Print elapsed time
    elapsed_time = end_time - start_time
    print(f"Time taken for Wu-Manber search: {elapsed_time:.6f} seconds")


Total words in the text: 135660
Pattern 'birds' found 10 time(s).
Pattern 'women' found 61 time(s).
Pattern 'horses' found 40 time(s).
Time taken for Wu-Manber search: 0.157471 seconds


___

# Conclusion

For the literary texts used in this exercies, the window shifting KMP and Boyer Moore algorithms outperformed the regex pattern search, while the exact search and trie algorithms were slower.  